In [1]:
import os
import numpy as np
import h5py
from scipy.io import loadmat
import PIL.Image
import nibabel as nib
import pandas as pd
import pickle
import time

import sys
sys.path.append("/home/prashant/BrainDiVE/modfit/code")

from utils import default_paths, roi_utils, prf_utils
from model_fitting import initialize_fitting
import gc

ModuleNotFoundError: No module named 'utils'

In [ ]:
exp_design_file = "/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_expdesign.mat"
exp_design = loadmat(exp_design_file)
image_order = exp_design['masterordering'].flatten() - 1 

In [ ]:
len(image_order)

30000

In [ ]:
np.min(image_order)

0

In [ ]:
image_order

array([ 625, 5012, 4849, ..., 4840, 7322, 9971], dtype=uint16)

In [ ]:
exp_design

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Jan  1 11:44:55 2019',
 '__version__': '1.0',
 '__globals__': [],
 'basiccnt': array([[447, 438, 423, 429, 426, 413, 420, 439, 423, 450, 462, 429, 421,
         432, 424, 438, 455, 443, 416, 425, 435, 460, 424, 451, 395, 449,
         452, 443, 456, 459, 465, 433, 459, 419, 431, 436, 444, 443, 436,
         424],
        [105, 102, 117, 102, 108, 116, 105, 109, 117,  90, 105, 117, 112,
         108, 109,  99, 101,  98, 107, 101, 102, 103, 115, 103, 104, 104,
         116, 101,  99, 102,  96, 109,  99, 119, 101, 103, 102,  98, 103,
         118],
        [ 31,  36,  31,  39,  36,  35,  40,  31,  31,  40,  26,  29,  35,
          34,  36,  38,  31,  37,  40,  41,  37,  28,  32,  31,  49,  31,
          22,  35,  32,  29,  31,  33,  31,  31,  39,  36,  34,  37,  36,
          30]], dtype=uint16),
 'masterordering': array([[ 626, 5013, 4850, ..., 4841, 7323, 9972]], dtype=uint16),
 'sharedix': array([[ 2951,  2991,  30

In [ ]:
x = exp_design["subjectim"]

In [ ]:
x

array([[ 2951,  2991,  3050, ..., 19472, 49508, 11124],
       [ 2951,  2991,  3050, ..., 29015, 40443,  7566],
       [ 2951,  2991,  3050, ..., 40627, 60549, 60491],
       ...,
       [ 2951,  2991,  3050, ..., 22828, 64982, 39213],
       [ 2951,  2991,  3050, ..., 68368, 68973, 16416],
       [ 2951,  2991,  3050, ..., 24254, 14948, 42093]], dtype=int32)

In [ ]:
a = np.repeat(np.arange(0, 5), 3)
a

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4])

In [ ]:
fn2load = "/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.nii.gz"
print('Loading from %s...'%fn2load)
values1 = nib.load(fn2load).get_fdata()
values = nib.load(fn2load).get_fdata().transpose((3, 0, 1, 2))
# values = load_from_nii(fn2load).transpose((3,0,1,2))
print('Raw data:')
print(values.dtype, np.min(values), np.max(values), values.shape)

betas = values.reshape((len(values), -1), order='C')

Loading from /media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.nii.gz...
Raw data:
float64 -32768.0 32767.0 (750, 81, 104, 83)


In [ ]:
print(values1.shape, values.shape, betas.shape)

(81, 104, 83, 750) (750, 81, 104, 83) (750, 699192)


In [7]:
subject = 1
fn2load = "/media/internal_8T/prashant/BrainDiVE/nsd/stimuli/Image_data_partitions.npy"
print()
print('loading train/holdout/val image list from %s'%fn2load)
partitions = np.load(fn2load, allow_pickle=True).item()
is_trn = partitions['is_trn'][:,subject-1]
is_holdout = partitions['is_holdout'][:,subject-1]
is_val = partitions['is_val'][:,subject-1]


loading train/holdout/val image list from /media/internal_8T/prashant/BrainDiVE/nsd/stimuli/Image_data_partitions.npy


In [6]:
print(is_trn, is_trn.shape)

[False False False ...  True  True  True] (10000,)


In [8]:
def get_subj_df(subject):
    """
    Get info about the 10,000 images that were shown to each subject.
    Note this is not the full ordered sequence of trials (which is 30,000 long)
    This is only the unique images 
    (matches what is in /user_data/mmhender/nsd_stimuli/stimuli/nsd/S1_stimuli....h5py)
    """
    exp_design_file = "/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_expdesign.mat"
    exp_design = loadmat(exp_design_file)
    subject_idx  = exp_design['subjectim']
    
    nsd_meta_file = "/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_stim_info_merged.pkl"
    with open(nsd_meta_file,'rb') as f:
        stim_info = pickle.load(f,encoding="latin1")
    
    ss=subject-1
    subject_df = stim_info.loc[subject_idx[ss,:]-1]

    return subject_df

In [19]:
ss = 7
subject_df = get_subj_df(ss)
val_image_inds = subject_df['shared1000']
trn_image_inds = ~subject_df['shared1000']
subject_df

,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.093750,2950,False,True,True,1,1,...,27566,2616,9716,27566,2616,9716,27566,2616,9716,27566
2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,2990,False,True,True,1,1,...,27711,18458,18697,27711,18458,18697,27711,18458,18697,27711
3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.000000,3049,False,True,True,1,1,...,6697,6299,6448,6697,6299,6448,6697,6299,6448,6697
3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,3077,False,True,True,1,1,...,4537,4289,4515,4537,4289,4515,4537,4289,4515,4537
3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,3146,False,True,True,1,1,...,26807,8087,8443,26807,8087,8443,26807,8087,8443,26807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62322,476569,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,62322,False,False,False,0,0,...,0,0,0,0,16047,25126,26116,0,0,0
47244,155754,train2017,"(0, 0, 0.0, 0.25)",0.142857,47244,False,False,False,0,0,...,0,0,0,0,20996,27976,28214,0,0,0
68367,499952,train2017,"(0, 0, 0.125, 0.125)",0.111111,68367,False,False,False,0,0,...,0,0,0,0,18337,18398,18606,0,0,0
68972,240175,train2017,"(0, 0, 0.125, 0.125)",0.000000,68972,False,False,False,0,0,...,0,0,0,0,1264,11935,17174,0,0,0


In [26]:
val_image_inds

2950      True
2990      True
3049      True
3077      True
3146      True
         ...  
1913     False
15168    False
19471    False
49507    False
11123    False
Name: shared1000, Length: 10000, dtype: bool

In [23]:
np.sum(val_image_inds)

1000

In [24]:
np.where(trn_image_inds)[0]

array([1000, 1001, 1002, ..., 9997, 9998, 9999])

In [33]:
inds_notval = np.where(trn_image_inds)[0]

inds_notval


array([1000, 1001, 1002, ..., 9997, 9998, 9999])

In [12]:
df["nsdId"]

2950      2950
2990      2990
3049      3049
3077      3077
3146      3146
         ...  
1913      1913
15168    15168
19471    19471
49507    49507
11123    11123
Name: nsdId, Length: 10000, dtype: int64

In [2]:
import pickle as pkl
import pandas as pd
 
# Load the pickled object
try:
    with open("/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_stim_info_merged.pkl", "rb") as f:
        data = pkl.load(f, encoding='latin1')  # Specify encoding to handle different character sets
except UnicodeDecodeError as e:
    print(f"Error loading pickle file: {e}")
    raise
 
# Convert the loaded object into a pandas DataFrame
df = pd.DataFrame(data)
 
# Save the DataFrame to a CSV file
df.to_csv(r'/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_stim_info_merged.csv', index=False)

In [3]:
csv_obj = pd.read_csv("/media/internal_8T/prashant/BrainDiVE/nsd_data/nsddata/experiments/nsd/nsd_stim_info_merged.csv")

In [4]:
csv_obj

,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
0,532481,val2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,0,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
1,245764,val2017,"(0, 0, 0.125, 0.125)",0.000000,1,False,False,False,0,0,...,0,0,0,0,13985,14176,28603,0,0,0
2,385029,val2017,"(0, 0, 0.125, 0.125)",0.000000,2,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
3,311303,val2017,"(0, 0, 0.16640625, 0.16640625)",0.125000,3,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,393226,val2017,"(0, 0, 0.125, 0.125)",0.133333,4,False,False,False,0,0,...,0,13720,22861,23023,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72995,518071,train2017,"(0, 0, 0.125, 0.125)",0.000000,72995,False,False,False,0,0,...,0,0,0,0,0,0,0,6083,11650,26531
72996,255930,train2017,"(0, 0, 0.125, 0.125)",0.125000,72996,False,False,False,0,0,...,0,0,0,0,0,0,0,10402,10434,10625
72997,255934,train2017,"(0, 0, 0.1, 0.1)",0.000000,72997,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
72998,518080,train2017,"(0.125, 0.125, 0, 0)",0.000000,72998,False,False,False,0,0,...,0,0,0,0,5585,11846,14495,0,0,0


In [20]:
all_coco = list(csv_obj["cocoId"])

In [30]:
a = np.unique(all_coco)

73000

In [35]:
current_box = list(csv_obj[csv_obj["cocoId"]==int(532481)]["cropBox"])
current_box 

['(0, 0, 0.1671875, 0.1671875)']

In [29]:
np.sum(csv_obj["cocoId"]==int(532481))

1

In [3]:
import numpy as np
import h5py

In [43]:
raw_path = "/media/internal_8T/prashant/BrainDiVE/NSD_zscored/subj_7.npy"
b = np.load(raw_path)

In [44]:
b.shape

(8470, 600210)

In [45]:
path = "/media/internal_8T/prashant/BrainDiVE/NSD_zscored/cortex/cortex_subj_7.npy"
data = h5py.File(path, 'r')
key = list(data.keys())

In [46]:
len(key)

8470

In [47]:
a = data[key[0]][:]

In [48]:
a.shape

(600210,)

In [49]:
key

['100012',
 '100274',
 '100283',
 '100337',
 '100354',
 '10039',
 '100396',
 '100404',
 '100438',
 '10046',
 '100468',
 '100517',
 '100579',
 '100599',
 '100611',
 '100717',
 '100798',
 '100827',
 '100863',
 '101059',
 '101069',
 '101106',
 '101265',
 '101283',
 '101357',
 '101401',
 '10149',
 '101620',
 '101643',
 '101780',
 '101862',
 '101952',
 '101969',
 '101985',
 '102004',
 '10229',
 '102297',
 '102353',
 '102357',
 '102405',
 '102460',
 '102529',
 '102594',
 '102655',
 '102663',
 '102670',
 '102724',
 '102754',
 '102755',
 '102815',
 '103030',
 '103086',
 '103098',
 '103151',
 '103163',
 '103236',
 '103257',
 '103306',
 '103366',
 '103383',
 '103539',
 '103556',
 '103613',
 '103699',
 '103873',
 '103890',
 '103969',
 '103970',
 '103978',
 '104001',
 '104044',
 '104081',
 '104125',
 '104126',
 '104177',
 '104191',
 '104326',
 '104368',
 '104406',
 '104502',
 '104588',
 '104589',
 '104711',
 '104817',
 '104837',
 '104953',
 '104973',
 '105014',
 '105113',
 '105452',
 '105529',
 '1